In [71]:
import pandas as pd
import os
import json
import numpy as np
import re

In [72]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\Anytime Lounge\AnytimeLoungeColMapping.json")
anytimeLounge_config = json.load(config_file)

In [73]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [74]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, additionalInfoIndex):
    
    allHoursOfOperation = []
    
    for i in range(0, len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}

        season = "All Season"
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
            
        additionalInfo = surveyData.iat[i, additionalInfoIndex]
        
        if not pd.isnull(additionalInfo) and additionalInfo.lower() != 'na' and additionalInfo.lower() != 'n/a':
            additionalInfoValue = {}
            additionalInfoValue["value"] = additionalInfo
            hoursOfOperationValue["AdditionalInformation"] = [additionalInfoValue]
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw):
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw):
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            print(startAndEndTime)
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [75]:
def determineBooleanValueForBar(dataFrame):
    val = ""
    if pd.notnull(dataFrame["ContainsBar"]) :
        if dataFrame["ContainsBar"] == "Yes":
            val = True
        elif dataFrame["ContainsBar"].strip() == "":
            val = ""
        else:
            val = False
    return val

In [76]:
## Getting the output File name
output_fileName = anytimeLounge_config['outputFileName']

common_column_indexes = []
column_names = []

## column index
columnIndexForAnyTimeLounge = []

## getting common column information and anytimeLounge course related column information from config file
common_column_count = len(anytimeLounge_config['commonColumnInformation'])
anytimeLounge_column_count = len(anytimeLounge_config['anytimeLoungeRelatedColumn'])

additionalInfoIndex = anytimeLounge_config['AdditionalInfoIndex']

## Getting column index from config json file
for i in range(0, common_column_count):
    common_column_indexes.append(anytimeLounge_config['commonColumnInformation'][i]['columnIndex'])
    column_names.append(anytimeLounge_config['commonColumnInformation'][i]['outputColumnName'])
    
for i in range(0, anytimeLounge_column_count):
    columnIndexForAnyTimeLounge.append(anytimeLounge_config['anytimeLoungeRelatedColumn'][i]['columnIndex'])
    column_names.append(anytimeLounge_config['anytimeLoungeRelatedColumn'][i]['outputColumnName'])
    
## getting Hours of Related column information from config file
hoursOfOperationInfo = anytimeLounge_config['HoursOfOperationInformation']


## Data for 1st Golf Course
anytimeLounge = surveyData.iloc[:,common_column_indexes + columnIndexForAnyTimeLounge]
anytimeLounge.columns = column_names

anytimeLounge['Index']=1

hrsOfOperation = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData, additionalInfoIndex)
anytimeLounge['HrsOfOpeartion'] = hrsOfOperation

## Adding these 3 information together
allGolfSurveyData = pd.concat([anytimeLounge])
allGolfSurveyData['Status'] = 'Active'
allGolfSurveyData['Type'] = 'Dining'


## deleting all the rows for which name is blank or, Na or, 0
dataExceptBlankName = allGolfSurveyData[(allGolfSurveyData['Name'].str.lower() != 'na') & (allGolfSurveyData['Name'].str.lower() != 'n/a') & (allGolfSurveyData['Name'].notnull()) & (allGolfSurveyData['Name'] != '0')]

dataExceptBlankName['Bar_Boolean'] = dataExceptBlankName.apply(determineBooleanValueForBar, axis=1)

dataExceptBlankName.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/Anytime Lounge/" + output_fileName)

07:00 AM#$09:00 PM
07:00 AM#$10:00 PM
11:00 AM#$09:00 PM
07:00 AM#$10:00 PM
07:00 AM#$08:00 PM
07:00 AM#$09:00 PM
07:00 AM#$10:00 PM
#$
06:00 AM#$06:00 PM
06:00 AM#$9:00 M
06:00 AM#$09:00 PM
07:00 AM#$08:00 PM
08:00 AM#$09:00 PM
11:00 AM#$09:00 PM
07:00 AM#$06:00 PM
07:00 AM#$09:00 PM
07:00 AM#$8:00 M
07:00 AM#$08:00 PM
11:00 AM#$08:00 PM
08:00 AM#$07:00 PM
11:00 AM#$09:00 PM
08:00 AM#$09:00 PM
11:00 AM#$07:00 PM
11:00 AM#$03:00 PM
11:00 AM#$09:00 PM
08:00 AM#$10:00 PM
07:00 AM#$06:00 PM
07:00 AM#$10:00 PM
07:00 AM#$08:00 PM
11:30 AM#$08:00 PM
11:00 AM#$09:00 PM
07:00 AM#$09:00 PM
11:00 AM#$10:00 PM
11:00 AM#$12:00 AM
11:00 AM#$09:00 PM
08:00 AM#$06:00 PM
10:00 AM#$06:00 PM
07:00 AM#$07:00 PM
07:00 AM#$10:00 PM
10:00 AM#$10:00 PM
10:00 AM#$12:00 AM
#$
07:00 AM#$09:00 PM
#$
06:00 AM#$06:00 PM
06:00 AM#$04:00 PM
06:00 AM#$09:00 PM
07:00 AM#$03:00 PM
05:00 PM#$10:00 PM
07:00 AM#$10:00 PM
#$
04:00 PM#$10:00 PM
11:00 AM#$09:00 PM
11:00 AM#$10:00 PM
05:30 PM#$11:00 PM
07:00 AM#$11:00 PM
07:0

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o